## ASTR 2100, Winter 2020

## Homework Assignment 1: Numerical integration (30 points)

### Distributed: January 17; Due Friday Jan 24 at 11:59pm 

#### Exercises you need to do for the assignment grade are under Exercise 1
#### Optional exercises are marked as extra-credit. Relative difficulty of each task in the exercise denoted by associated points

#### Please submit your work in the form of Jupyter notebook via Canvas. Make sure that all of the cells in the notebook have output that you want, when you run all cells just before the submission


#### Jupyter notebooks with class notes and assignments: https://github.com/a-kravtsov/a211w20

#### Slack workspace for this class: https://a211w20.slack.com


#### Instructor: Andrey Kravtsov (kravtsov@uchicago.edu)

#### TAs: Dimitrios Tanoglidis (dtanoglidis@uchicaago.edu), Georgios Zakharegkas (gzakharegkas@uchicago.edu)

### Note: 

The required assignment is Exercises 1a-1c. You can also do exercises 2 and/or 3 for extra credit points, if you want. You can also do the Exercise 2 *instead of* 1a-1c, in this case you will receive 10 extra credit points. 

## Exercise 1 

**Background.** We discussed numerical integration with the trapezoidal scheme and the errors associated with it. Notebook <tt>02_integration_approximation.ipynb</tt> contains routines that compute the luminosity distance $d_L$ using trapezoidal scheme, as well as Romberg method implemented in the SciPy's integration routine <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.romberg.html"><tt>scipy.integrate.romberg</tt></a>. This routine that uses my trapezoidal routine is reproduced at the bottom of this notebook. In this exercise, I would like you to replace my routine implementing trapezoidal scheme with your own routine that estimates the integral to the 4th order of accuracy (i.e., fractional error scaling as $\propto h^4$, where $h$ is step size used) using the trick of elimination of the leading error term.   

In other words, your routine should implement computation of $R_1(h) = [4T(h/2)-T(h)]/3$, where $T(h)$ and $T(h/2)$ are integral estimates with the trapezoidal schem with step sizes $h$ and $h/2$, respectively. This is actually done in my routine <tt>trapzd2</tt> in the <tt>02_integration_approximation.ipynb</tt> notebook, but in there it is done in a computationally inefficient way. Namely, when it computes $T(h/2)$ via a call to <tt>trapzd</tt> it does the same work as in computation of $T(h)$, because half of the points at which $f(x)$ is evaluated in $T(h/2)$ are the same as in the calculation of $T(h)$. The routine you implement in this exercise should avoid this redundancy. The note below explains how do this.


### A more efficient calculation of $R_1(h)$

If we denote $h_j = (b-a)/2^j$ and trapezoidal scheme with step $h_j$ as $T_j$, then it is easy to show that trapezoidal scheme for $j=1, 2, \ldots$ can be written as

$$T_j = \frac{1}{2} T_{j-1} + h_j\sum\limits_{i=1}^{2^{j-1}}f(a+(2i-1)h_j).$$

This expression allows to reuse the previous computations used to estimate $T_{j-1}$ in computing $T_j$. Taking this into account, the computation of $R_1(h)$ described above, can be written as 

$$R_1(h) = \frac{1}{3}\left[4T(h/2)-T(h)\right]=\frac{1}{3}\left[T(h)+2h\sum\limits_{i=1}^{2^{k-1}}f\left(a+(2i-1)\frac{h}{2}\right)\right],$$

where $k=\log_2[2(b-a)/h]$ and it is expected that the total number of sub-intervals of $[a,b]$ is a power of 2. The second term in the right hand side above, only adds calculations required for $R_1(h)$ that have not been yet done during calculation of $T(h)$ and thus avoids the redudancy.

Note that numpy function for $\log_2$ is <tt>np.log2</tt>

Note also that you can generate a vector of points $a+(2i-1)h_j$  at which to evaluate function using <tt>np.arange</tt>

#### Estimating absolute and relative errors of the integral estimate

Given that we compute $T(h)$ during calculation of $R_1(h)$ (in the algorithm jargon it is said that computation of $T(h)$ is *embedded* in the calculation of $R_1(h)$), we can use these two estimates of the integral of different accuracy to estimate the error of the integration for the current step size $h$, *for the cases when we do not know exact value of the integral* (which is in all practical applications, because if we knew the value of the integral there would be no need to evaluate it using a numerical integration method). 

Namely, the absolute error can be estimated as 
$\epsilon_{\rm abs}=\vert R_1(h)-T(h)\vert$
and fractional error as $\epsilon_{\rm r}=\vert1- T(h)/R_1(h)\vert$. Note that for the fractional error, we should guard against the cases when $R_1(h)=0$. 

### What you are learning in this exercise

* Method for efficiently calculating estimates of $R_m$ in the Romberg iterations. This can be used as a basis for implementation of the full Romberg integration in the optional exercise below. 

* Testing your implementation and making plots. 

* Improving your understanding of the truncation errors of the trapezoidal scheme. 

* Learning how to implement error control in your numerical routine using absolute and/or relative error tolerance thresholds. 

* Practice using vectorized operations with numpy. 


**Task 1a. (15 points)**
Implement a routine to compute $R_1(h)$ scheme using the efficient method desribed above and check it for a test function for which you know the integral exactly. Plot fractional error as a function of step size and demonstrate that the error in your implemented routine indeed scales as $\propto h^4$.

Implement capability of passing a pointer to a list of arguments <tt>*args</tt> to function that is integrated. Use my routines at the bottom of this notebook as an example. If these examples are not sufficiently clear, you can read more about this way of passing arguments <a href="https://www.python-course.eu/python3_passing_arguments.php">here</a>.

**Task 1b. (10 points)** Use the fact that $T(h)$ and $R_1(h)$ provide 2 estimates of the integral of different accuracy, implement a routine similar to above that can use the difference between $R_1(h)$ and $T(h)$ for steps of decreasing accuracy to figure out what step $h$ to use to achieve input threshold values of the absolute tolerance <tt>atol</tt> and fractional accuracy <tt>rtol</tt>. Test your routine by computing an integral with different tolerance values and examining the fractional error relative to exact result. 

**Task 1c. (5 points)**  Implement routine computing luminosity distance $d_L$ using the integration routine developed in the task 1a and 1b. For uniformity and ease of testing your routine, please keep the interfaces of your routines similar to my routines. However, if you implement 1b, add input parameters for <tt>atol</tt> and fractional accuracy <tt>rtol</tt>.

If you do not manage to get your implementation working, use my routine below for subsequent exercise 

**Task 1d. Extra-credit (10 points)** Implement a routine for computing $d_L$ based on the trapezoidal integration scheme that is 4th order accurate (i.e has fractional error scaling as $\propto h^4$) *without* using the formula $R_1(h)=[4T(h/2)-T(h)]/3$. Test your routine and demonstrate that its fractional error scales as $\propto h^4$ using a plot. 

## Exercise 2 (extra-credit)

Implement a routine performing Romberg integration of input function $f(x)$ and integration interval $[a,b]$ to a specified *fractional* error tolerance. Implement the integration routine in such a way that one can pass a list of parameter values of arbitrary size to $f(x)$ using <tt>*args</tt> pointer as in my routines below. 

The Romberg integration starts either with a step size equal to the entire integration interval ($h_0=b-a$) to compute $R_0(h_0)=T(h_0)$ and proceeds to estimate $R_m$ for $m=1,2,\ldots$ using equation derived in class: 

$$
R_{m+1}(h)=\frac{4^{m+1} R_{m}(h/2)-R_{m}(h)}{4^{m+1}-1}.
$$

One can implement this via recursive calls to the same function, but a much more efficient is to reuse computations done for $R_m(h)$ in computing $R_m(h/2)$. 


### What you are learning in this exercise

* Efficient implementation of the Romberg integration method to do integration to specific error tolerance level (error control).

**Task 2a. (35 points)** Implement a routine that estimates integral $\int_a^b f(x)dx$ using Romberg integration method with error better than specified tolerance level, as in the task 1b above, but using $R_{m+1}$ and $R_{m}$ instead of $R_1$ and $T$. Test your routine by computing $$\int_0^1 e^x dx$$ for different tolerance levels and comparing results with the 
true value of the integral. 

Namely, implement the function with the following inputs and outputs:

    def romberg(func, a, b, atol, rtol, *args):
        """
        Parameters:
        --------------------------------
        func - python function object
               function to integrate
        a, b - floats
               integration interval
        rtol - float 
               fractional tolerance of the integral estimate
        args - list
               a list of parameters to pass to func
               parameters must be in order and number expected by func
               
        Returns:
        ---------------------------------
        I    - float
               estimate of the integral for input f, [a,b] and rtol
        err  - float 
               estimated fractional error of the estimated integral
        """


**Task 2b. (5 points)** Implement a routine to compute luminosity distance $d_L$ using the integration routine developed in the task 2a. For uniformity and ease of testing your routine, please keep the interfaces of your routines similar to my routines. However, if you implement 1b, add input parameters for <tt>atol</tt> and fractional accuracy <tt>rtol</tt>.

## Exercise 3 (extra-credit, 15 points)

Parallelize your Romberg integration routine with message passing interface (MPI) library by splitting integration interval into independent segments and applying Romberg integration on each one independently. I would be happy to go over basics of using MPI for basic parallelization with you, if you choose to do this exercise. You can also check out this tutorial on <a href="https://mpi4py.readthedocs.io/en/stable/tutorial.html">mpi4py</a>. You will need to have mpi4py installed on your laptop for this exercise or on a machine were you can run your code. Note that mpi4py is already installed if you installed the Ananconda python distribution. 

Test your implementation and check how its performance scales with a different number of processors. 

You can find example of parallel computation of integral with trapezoidal quadrature  <a href="http://materials.jeremybejarano.com/MPIwithPython/collectiveCom.html">here</a> that can be used as a template. 


## My code to compute $d_L$ 


### $d_L$ for models with $\Omega_\Lambda=0$. 

In [1]:
import numpy as np

clight = 2.99792458e5 # c in km/s

# function that works only for models with OmL = 0
def d_L_simple(z, H0, Om0):
    q0 = 0.5 * Om0
    q0z = q0 * z
    return clight * z/H0 * (1. + (z-q0z) / (1. + q0z + np.sqrt(1. + 2.*q0z)))  

### $d_L$ for general models with $\Omega_\Lambda\ne 0$ 

$$d_c =\int\limits_0^z \frac{dz}{E(z)}.$$

where $\Omega_k = 1-\Omega_{\rm m0} - \Omega_\Lambda$ and $E(z)=\sqrt{\Omega_{\rm m0}(1+z)^3+\Omega_k(1+z)^2+\Omega_\Lambda}.$

$$
d_L(z,\Omega_0,\Omega_\Lambda) = \frac{c}{H_0}\,(1+z)\,\left\{
\begin{array}{ll}
\frac{1}{\sqrt{\Omega_k}}\,\sinh\left[\sqrt{\Omega_k}\,d_{\rm c}\right] & {\rm for}~\Omega_k>0 \\
d_{\rm c} & {\rm for}~\Omega_k=0 \\
\frac{1}{\sqrt{|\Omega_k|}}\,\sin\left[\sqrt{|\Omega_k|}\,d_{\rm c}\right] & {\rm for}~\Omega_k<0
\end{array}
\right.
$$


In [2]:
def dl_func(z, H0, Om0, OmL, Omk):
    z1 = 1.0 + z; z12 = z1 * z1
    return 1.0 / np.sqrt(z12*(Om0*z1 + Omk) + OmL)

Adding capability to pass a vector of arguments to the integrand function to trapzd routine using <tt>*args.</tt>

$\star$ in <tt>*args</tt> indicates that what is passed is a *pointer* to memory location of the start of args, not the values of args elements themselves. The trapzd routine simply passes *args to function, which then interprets the contant as the parameters it expects.

In [3]:
def trapzd(func, a, b, hstep, *args):
    """
    integration using trapezoidal scheme
    
    Parameters:
    -----------
    func: python function object - function to integrate
    a, b: floats - integration interval
    hstep: float - step size to use
    *args: pointer to argument list to pass to dl_func
    
    Returns:
    --------
    float - value of the integral estimated using trapezoidal integration    
    """
    bma = np.abs(b-a)
            
    nstep = np.int(bma / hstep) 
    if nstep == 1:
        return 0.5*(func(a, *args) + func(b, *args)) * hstep
    else: 
        xd = a + np.arange(nstep) * hstep # np.arange creates a vector of values from 0 to nstep-1
        return (0.5*(func(a, *args) + func(b, *args)) + np.sum(func(xd[1:], *args))) * hstep


In [4]:
def _dc(z1, z2, hstep, *args):
    """
    helper function to compute D_c from z1 to z2 by numerical integration using trapzd
    
    Parameters: 
    -----------
    z1, z2: floats = redshift integration limits
    hstep: float = step size for trapezoidal integration
    *args: pointer to argument list to pass to dl_func
    
    Returns:
    --------
    float - value of the integral estimated using trapezoidal integration
    """
    return trapzd(dl_func, z1, z2, hstep, *args)

In [5]:
def d_L(z, H0, Om0, OmL, hstep=1.e-8):
    """
    routine computing luminosity distance from z=0 to z 
    using helper routine for comoving distance _dc
    
    Parameters:
    -----------
    z: float - redshift to which to compute d_L
    H0: float - Hubble constant in units of km/s/Mpc
    Om0: float - dimensionless mean matter density in the universe, >= 0
    OmL: float - dimensionless mean vacuum energy density in the universe
    hstep: float - step size to use
     
    Returns:
    --------
    float - value of d_L(z, H0, Om0, OmL) 
            estimated using trapezoidal integration with step size hstep 
    """
    if 1.0+OmL == 1.0: # note that I exploit the roundoff error in testing whether OmL is consistent with zero
        return d_L_simple(z, H0, Om0) # if OmL=0, use simple formula without integration
    else: # otherwise use numerical integration to compute distance 
        dH = clight / H0 
        Omk = 1.0 - Om0 - OmL
        args = [H0, Om0, OmL, Omk]
        dc = _dc(0., z, hstep, *args)
        if 1.0 + Omk == 1.0:
            return dH * dc * (1.0 + z)
        else:
            sqrOmk = np.sqrt(np.abs(Omk))
            if Omk > 0.:
                return dH * np.sinh(dc*sqrOmk) * (1.0 + z) / sqrOmk 
            else:
                return dH * np.sin(dc*sqrOmk) * (1.0 + z) / sqrOmk
            
        return dL

In [7]:
H0, Om0, OmL = 70., 0.3, 0.7;
z = 2.0

hstep = 1.e-6

dLtrap    = d_L(z, H0, Om0, OmL, hstep=hstep)

print("d_L(z=%.2f, Om0=%.2f, OmL=%.2f, H0=%.1f) = %.4g Mpc"%(z, Om0, OmL, H0, dLtrap))


d_L(z=2.00, Om0=0.30, OmL=0.70, H0=70.0) = 1.554e+04 Mpc
